<a href="https://colab.research.google.com/github/denis911/image-classification-transfer-learning-pytorch-keras-comparison/blob/main/HW8_hair_type_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the dataset

In [1]:
!nvidia-smi # nvidia system information for T4 Tesla GPU

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# We'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from Kaggle and slightly rebuilt.

# RUN ONCE:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip > /dev/null

--2025-12-01 20:40:11--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T21%3A27%3A41Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T20%3A27%3A23Z&ske=2025-12-01T21%3A27%3A41Z&sks=b&skv=2018-11-09&sig=SxqcQTPcTu74bB0rCkfD5qRH1qEgGyClHR2%2FlsYDPr4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyMzQxMSwibmJmIjoxNzY0NjIxNjExLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [3]:
# RUN ONCE
# unzip dataset into Colab home folder and check:
!unzip data.zip > /dev/null
# > /dev/null to prohibit long output, then check files manually

In [4]:
# Reproducibility
# Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

# Therefore we suggest to set the random number seed generators by:

import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



## Model

In [7]:
# Model to classify curly vs straight hair from the photo
# We will use Convolutional Neural Network (CNN) and PyTorch.

# TASK - develop the model with following structure:

# The shape for input should be (3, 200, 200) (channels first format in PyTorch)
# Next, create a convolutional layer (nn.Conv2d):
# Use 32 filters (output channels)
# Kernel size should be (3, 3) (that's the size of the filter), padding = 0, stride = 1
# Use 'relu' as activation
# Reduce the size of the feature map with max pooling (nn.MaxPool2d)
# Set the pooling size to (2, 2)
# Turn the multi-dimensional result into vectors using flatten or view
# Next, add a nn.Linear layer with 64 neurons and 'relu' activation
# Finally, create the nn.Linear layer with 1 neuron - this will be the output
# The output layer should have an activation - use the appropriate activation for the binary classification case - SIGMOID

# As optimizer use torch.optim.SGD with the following parameters:

# torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

import torch
import torch.nn as nn
import torch.nn.functional as F

class HairClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)     # <-- raw logits, no sigmoid
        # our loss will be in the training nn.BCEWithLogitsLoss()

        return x


# Q1 - we use loss  nn.BCEWithLogitsLoss()


In [9]:
# Q2 = Total params: 20,073,473
# What's the total number of parameters of the model?
# In PyTorch, you can find the total number of parameters using:

# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
model = HairClassifier()
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------
